Essa parte foi toda realizada utilizando o Google Colab, por isso alguns comandos de instalação podem estar ligeiramente diferentes de como seriam no VSCode

Importando a Biblioteca

In [11]:
!pip install pyspark==3.3.1

In [2]:
from pyspark.sql import SparkSession
from google.colab import files

Upload da base extraída do SQL

In [3]:
uploaded = files.upload()

Saving COVID 3 trim 2020 SP Cap.csv to COVID 3 trim 2020 SP Cap.csv


In [4]:
# Criar uma sessão Spark
spark = SparkSession.builder \
    .appName("Base COVID") \
    .getOrCreate()


In [5]:
# Ler o CSV
import io
file_name = next(iter(uploaded))
df = spark.read.csv(file_name, header=True, inferSchema=True)


Renomeado as colunas para facilitar o entendimento

In [6]:
df_renomeado = df \
    .withColumnRenamed("A003", "Sexo") \
    .withColumnRenamed("B008", "Fez o teste de covid") \
    .withColumnRenamed("Result_teste_covid", "Resultado do teste") \
    .withColumnRenamed("B007", "Possui plano de saúde") \
    .withColumnRenamed("B002", "Buscou atendimento médico") \
    .withColumnRenamed("B011", "Isolamento social") \
    .withColumnRenamed("B005", "Precisou ser internado ao visitar um hospital") \
    .withColumnRenamed("B0011", "Febre") \
    .withColumnRenamed("B0012", "Tosse") \
    .withColumnRenamed("B0013", "Dor de garganta") \
    .withColumnRenamed("B0014", "Dificuldade para respirar") \
    .withColumnRenamed("B0015", "Dor de cabeça") \
    .withColumnRenamed("B0016", "Dor no peito") \
    .withColumnRenamed("B0017", "Nausea") \
    .withColumnRenamed("B0018", "Nariz entupido ou escorrendo") \
    .withColumnRenamed("B0019", "Fadiga") \
    .withColumnRenamed("B00110", "Dor nos olhos") \
    .withColumnRenamed("B00111", "Perda de cheiro ou sabor") \
    .withColumnRenamed("B00112", "Dor muscular") \
    .withColumnRenamed("B00113", "Diarreia")

DataFrame com as colunas renomeadas

In [9]:
df_renomeado.show()

+---+-------+----+--------------------+---------------------+-------------------------+-----------------+---------------------------------------------+-----+-----+---------------+-------------------------+-------------+------------+------+----------------------------+------+-------------+------------------------+------------+--------+---+------------------+
| UF|CAPITAL|Sexo|Fez o teste de covid|Possui plano de saúde|Buscou atendimento médico|Isolamento social|Precisou ser internado ao visitar um hospital|Febre|Tosse|Dor de garganta|Dificuldade para respirar|Dor de cabeça|Dor no peito|Nausea|Nariz entupido ou escorrendo|Fadiga|Dor nos olhos|Perda de cheiro ou sabor|Dor muscular|Diarreia|Mes|Resultado do teste|
+---+-------+----+--------------------+---------------------+-------------------------+-----------------+---------------------------------------------+-----+-----+---------------+-------------------------+-------------+------------+------+----------------------------+------+-----

Agora que as colunas foram renomeadas, começaremos a responder as questões.

# - Houve um aumento entre Julho e Setembro de testes positivos para covid na capital?

In [10]:
from pyspark.sql import functions as F

# Filtrar apenas os resultados positivos (1 Positivo, 2 negativo)
df_positivos = df_renomeado.filter(df_renomeado['Resultado do teste'] == 1)


In [11]:
# Contar o número de testes positivos por mês
resultados_por_mes = df_positivos.groupBy("Mes").agg(F.count("*").alias("Total_Positivos"))
resultados_por_mes.show()


+---+---------------+
|Mes|Total_Positivos|
+---+---------------+
|  9|            200|
|  8|            182|
|  7|            272|
+---+---------------+



In [18]:
# Coletando os resultados em lista
resultados_lista = resultados_por_mes.collect()

# Criando uma lista para armazenar as porcentagens
percentuais = []

# Adicionando o percentual de Julho (0% porque é o primeiro mês)
if len(resultados_lista) > 0:
    percentuais.append(("Julho", "Julho", 0.0))  # Usar float

# Mapeamento de meses
mes_mapping = {
    7: "Julho",
    8: "Agosto",
    9: "Setembro"
}

# Calculando os percentuais para Agosto e Setembro
for i in range(1, len(resultados_lista)):
    mes_anterior = mes_mapping.get(resultados_lista[i-1][0], "Desconhecido")  # Mapear mês anterior
    positivos_anterior = resultados_lista[i-1][1]
    positivos_atual = resultados_lista[i][1]
    mes_atual = mes_mapping.get(resultados_lista[i][0], "Desconhecido")  # Mapear mês atual

    if positivos_anterior > 0:
        percentual = float((positivos_atual - positivos_anterior) / positivos_anterior * 100)  # Garantir que é float
    else:
        percentual = None  # Para não dividir por zero

    percentuais.append((mes_anterior, mes_atual, percentual))

# Criar um DataFrame para as porcentagens
df_percentuais = spark.createDataFrame(percentuais, ["Mes_Anterior", "Mes_Atual", "Percentual"])
df_percentuais.show()


+------------+---------+-----------------+
|Mes_Anterior|Mes_Atual|       Percentual|
+------------+---------+-----------------+
|       Julho|    Julho|              0.0|
|    Setembro|   Agosto|             -9.0|
|      Agosto|    Julho|49.45054945054945|
+------------+---------+-----------------+



Respondendo nossa primeira pergunda, sim. Houve um aumento de casos entre Julho e Setembro. O numero de positivos em Agosto é menor do que os de Julho, isso pode ter ocorrido em decorrencia de medidas de distanciamento, higienização e uso de máscara. O aumento de casos em Setembro pode ser um reflexo da necessidade da população de voltar a se descolar, o aumento de pessoas visitando hospitais e se contaminando ou um reflexo do delay entre a realização do teste e o recebimento do resultado.

# - Houve diferença significativa entre o número de homens e mulheres que testaram positivo?

In [9]:
from pyspark.sql import functions as F

# Filtrar os dados para contar os positivos por sexo
df_sexo_positivos = df_renomeado.filter(df_renomeado['Resultado do teste'] == 1) \
    .groupBy("Sexo") \
    .agg(F.count("*").alias("Total_Positivos"))

df_sexo_positivos.show()

+----+---------------+
|Sexo|Total_Positivos|
+----+---------------+
|   1|            283|
|   2|            371|
+----+---------------+



In [10]:
# Coletar os resultados
resultados_sexo = df_sexo_positivos.collect()

# Criar um dicionário para armazenar os dados
totais_sexo = {1: 0, 2: 0}  # 1 = Homem, 2 = Mulher

for row in resultados_sexo:
    totais_sexo[row[0]] = row[1]

# Total geral de positivos
total_positivos = sum(totais_sexo.values())

# Calcular porcentagens
porcentagens_sexo = {sexo: (total / total_positivos) * 100 for sexo, total in totais_sexo.items()}

print("Totais de Positivos:", totais_sexo)
print("Porcentagens de Positivos:", porcentagens_sexo)

Totais de Positivos: {1: 283, 2: 371}
Porcentagens de Positivos: {1: 43.27217125382263, 2: 56.727828746177366}


Então sim, houve uma diferença na quantidade de testes positivos entre homens e mulheres. Foram aproximadamente 31% a mais de mulheres infectadas em comparação com homens durante esse período. Isso pode ser explicado por questões históricas e sociais já ainda hoje as mulheres exercem um papel central na gestão da família, exposição para compras da manutenção do lar e cuidado de crianças, o que pode as ter expostas a situações de risco. Outro fator importante a ser considerado é que historicamente os homens tendem a ser mais negligentes com a própria saúde, o que pode levá-los a ignorar sintomas e sequer realizar os testes.


# - Porcentagem que buscou atendimento médico após apresentar algum sintoma

Agora vamos fazer o seguinte levantamento. Das pessoas que apresentaram sintomas, quantas de fato buscaram algum tipo de atendimento médico?

In [12]:
from pyspark.sql import functions as F

# Criei uma condição para verificar se pelo menos um dos sintomas foi relatado como "sim"
condicao_sintomas = (
    (df_renomeado['Febre'] == 1) |
    (df_renomeado['Tosse'] == 1) |
    (df_renomeado['Dor de garganta'] == 1) |
    (df_renomeado['Dificuldade para respirar'] == 1) |
    (df_renomeado['Dor de cabeça'] == 1) |
    (df_renomeado['Dor no peito'] == 1) |
    (df_renomeado['Nausea'] == 1) |
    (df_renomeado['Nariz entupido ou escorrendo'] == 1) |
    (df_renomeado['Fadiga'] == 1) |
    (df_renomeado['Dor nos olhos'] == 1) |
    (df_renomeado['Perda de cheiro ou sabor'] == 1) |
    (df_renomeado['Dor muscular'] == 1) |
    (df_renomeado['Diarreia'] == 1)
)

# Filtrar o DataFrame
df_filtrado = df_renomeado.filter(condicao_sintomas)


In [13]:
# O próximo passo é contar quantas pessoas buscaram atendimento médico
df_atendimento = df_filtrado.groupBy("Buscou atendimento médico").agg(F.count("*").alias("Total"))

df_atendimento.show()

+-------------------------+-----+
|Buscou atendimento médico|Total|
+-------------------------+-----+
|                        1|  417|
|                        2| 1901|
+-------------------------+-----+



In [15]:
# Coletando os resultados
resultados_atendimento = df_atendimento.collect()

# Total de pessoas que relataram sintomas
total_sintomas = sum(row[1] for row in resultados_atendimento)

# Calcular porcentagens
porcentagens_atendimento = {1: 0, 2: 0}  # 1 = Sim, 2 = Não

for row in resultados_atendimento:
    porcentagens_atendimento[row[0]] = (row[1] / total_sintomas) * 100

print("Porcentagem que buscou atendimento médico: {:.2f}%".format(porcentagens_atendimento[1]))
print("Porcentagem que não buscou atendimento médico: {:.2f}%".format(porcentagens_atendimento[2]))

Porcentagem que buscou atendimento médico: 17.99%
Porcentagem que não buscou atendimento médico: 82.01%


# - Ao visitar um hospital, precisou ser internado?

In [18]:
# Filtrar os dados para considerar apenas as pessoas que visitaram hospitais
df_hospital = df_renomeado.filter(df_renomeado['Precisou ser internado ao visitar um hospital'] != 2)

# Contar as respostas para a coluna "Precisou ser internado"
df_internacao = df_hospital.groupBy("Precisou ser internado ao visitar um hospital").agg(F.count("*").alias("Total"))

df_internacao.show()


+---------------------------------------------+-----+
|Precisou ser internado ao visitar um hospital|Total|
+---------------------------------------------+-----+
|                                            1|   13|
|                                            3|    5|
+---------------------------------------------+-----+



In [19]:
# Coletar os resultados
resultados_internacao = df_internacao.collect()

# Total de pessoas que visitaram hospitais
total_hospital = sum(row[1] for row in resultados_internacao)

# Calcular porcentagens
porcentagens_internacao = {1: 0, 2: 0, 3: 0}  # 1 = Sim, 2 = Não, 3 = Não foi atendido

for row in resultados_internacao:
    porcentagens_internacao[row[0]] = (row[1] / total_hospital) * 100

print(f"Porcentagem que precisou ser internado: {porcentagens_internacao[1]:.2f}%")
print(f"Porcentagem que não precisou ser internado: {porcentagens_internacao[2]:.2f}%")
print(f"Porcentagem que não foi atendido: {porcentagens_internacao[3]:.2f}%")


Porcentagem que precisou ser internado: 72.22%
Porcentagem que não precisou ser internado: 0.00%
Porcentagem que não foi atendido: 27.78%


Das pessoas que foram ao hospital e conseguiram ser atendidas, 72.22% precisou ser internada por pelo menos um dia, enquanto os outros 27.78% não conseguiram atendimento. Isso reforça o ponto de que as pessoas evitaram buscar atendimento a menos que os sintomas estivessem com maior gravidade, e que os próprios estabelecimento de saúde também estavam priorizando apenas os casos urgentes.

# - Porcentagem que possuía algum plano de saúde

In [21]:
# Contar as respostas para a coluna "Possui plano de saúde"
df_plano_saude = df_renomeado.groupBy("Possui plano de saúde").agg(F.count("*").alias("Total"))

df_plano_saude.show()


+---------------------+-----+
|Possui plano de saúde|Total|
+---------------------+-----+
|                    1|14360|
|                    9|  466|
|                    2|16383|
+---------------------+-----+



In [22]:
# Coletar os resultados
resultados_plano_saude = df_plano_saude.collect()

# Total de pessoas
total_pessoas = sum(row[1] for row in resultados_plano_saude)

# Calcular porcentagens
porcentagens_plano_saude = {1: 0, 2: 0}  # 1 = Sim, 2 = Não

for row in resultados_plano_saude:
    porcentagens_plano_saude[row[0]] = (row[1] / total_pessoas) * 100

print(f"Porcentagem que possui plano de saúde: {porcentagens_plano_saude[1]:.2f}%")
print(f"Porcentagem que não possui plano de saúde: {porcentagens_plano_saude[2]:.2f}%")


Porcentagem que possui plano de saúde: 46.01%
Porcentagem que não possui plano de saúde: 52.49%


# - Como foi o isolamento social duante o período analisado.

In [23]:
# Novamente vou contar as respostas para a coluna "Isolamento Social"
df_isolamento = df_renomeado.groupBy("Isolamento social").agg(F.count("*").alias("Total"))

df_isolamento.show()


+-----------------+-----+
|Isolamento social|Total|
+-----------------+-----+
|                1|  797|
|                3|14187|
|                9|  598|
|                4| 6639|
|                2| 8988|
+-----------------+-----+



In [24]:
# Coleta de resultados
resultados_isolamento = df_isolamento.collect()

# Total de pessoas
total_pessoas_isolamento = sum(row[1] for row in resultados_isolamento)

# Porcentagens
porcentagens_isolamento = {1: 0, 2: 0, 3: 0, 4: 0}  # 1, 2, 3, 4 conforme os valores

for row in resultados_isolamento:
    porcentagens_isolamento[row[0]] = (row[1] / total_pessoas_isolamento) * 100

print(f"Porcentagem que não fez restrição social: {porcentagens_isolamento[1]:.2f}%")
print(f"Porcentagem que reduziu o contato: {porcentagens_isolamento[2]:.2f}%")
print(f"Porcentagem que só saiu em caso de necessidade: {porcentagens_isolamento[3]:.2f}%")
print(f"Porcentagem que ficou rigorosamente em casa: {porcentagens_isolamento[4]:.2f}%")


Porcentagem que não fez restrição social: 2.55%
Porcentagem que reduziu o contato: 28.80%
Porcentagem que só saiu em caso de necessidade: 45.46%
Porcentagem que ficou rigorosamente em casa: 21.27%


* 2.55% dessa amostra não fez nenhuma restrição social durante esse período,
continuando sua rotina normalmente como antes da pandemia
* 28.80% reduziu o contato com outras pessoas em algum nível, mas continuou saindo de casa para atividades não essenciais, trabalho e/ou recebendo visitas em casa
* 45.46% só saiu em caso de necessidade básica
* 21.27% se manteve completamente isolado em casa

Pelos resultados podemos concluir que nesse primeiro momento o comportamento geral foi de aderir (com algumas adaptações) as indicações de isolamento social, mas sem abrir mão de algumas atividades normais.

Mas isso somado ao fato de que uma minoria ainda manteve sua rotina sem nenhuma alteração evidencia a importância de mais campanhas de conscientização sobre a importância do isolamento e os riscos do seu descumprimento.